In [ ]:
import pandas as pd
import pathlib
import glob
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.optimizers import RMSprop, Adam
from keras.layers import Dense,Flatten , Conv2D , MaxPooling2D , Dropout, Rescaling
from keras.models import Model , Sequential
from keras.callbacks import EarlyStopping
from keras.utils import image_dataset_from_directory

In [ ]:
data_dir = '../data/'
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    labels='inferred',
    label_mode='categorical',
    image_size=[224, 224],
    seed=123,
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    labels='inferred',
    label_mode='categorical',
    image_size=[224, 224],
    seed=123,
    interpolation='nearest',
    batch_size=64,
    shuffle=False,
)

In [ ]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = (train_ds.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE))
val_ds = (val_ds.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE))

### MobileNet Freezing 7

In [ ]:
from keras.applications.mobilenet import MobileNet
transfer_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224,224,3))

num = 1
for layer in transfer_model.layers:
  if num > 7:
    layer.trainable = True
  num += 1

In [ ]:
model = Sequential()

model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(224,224,3)))

model.add(transfer_model)

model.add(Conv2D(16, 3, padding='same', activation='relu')) 
model.add(MaxPooling2D(padding='same')) 
model.add(Dropout(0.2))

model.add(Conv2D(32, 3, padding='same', activation='relu')) 
model.add(MaxPooling2D(padding='same'))

model.add(Conv2D(64, 3, padding='same', activation='relu')) 
model.add(MaxPooling2D(padding='same')) 
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, activation='relu')) 
model.add(Dense(33 , activation = 'softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds,batch_size=64,epochs=20,validation_data=val_ds)

In [ ]:

model.evaluate(train_ds),model.evaluate(val_ds)

In [ ]:
plt.style.use('fivethirtyeight')
fig,axes=plt.subplots(nrows=1, ncols=2, figsize=(20,10))
axes[0].plot(history.history["loss"], 'r', label='Training loss')
axes[0].plot(history.history["val_loss"],'g',label='Validation loss' )
axes[0].set_title('Mobilenet Model (Freezing before 8) Training and Validation Loss')
axes[0].set_xlabel('Epochs')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[1].plot (history.history["accuracy"],'r',label= 'Training Accuracy')
axes[1].plot (history.history["val_accuracy"],'g',label= 'Validation Accuracy')
axes[1].set_title('Mobilenet Model (Freezing before 8) Training and Validation Accuracy')
axes[1].set_xlabel('Epochs')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
plt.savefig("/content/drive/MyDrive/Colab Notebooks/Mobilenet_Freezing_8_accuracy.jpg")

### MobileNet Freezing 14

In [ ]:
from keras.applications.mobilenet import MobileNet
transfer_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224,224,3))

num = 1
for layer in transfer_model.layers:
  if num > 14:
    layer.trainable = True
  num += 1

In [ ]:
model = Sequential()

model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(224,224,3)))

model.add(transfer_model)

model.add(Conv2D(16, 3, padding='same', activation='relu')) 
model.add(MaxPooling2D(padding='same')) 
model.add(Dropout(0.2))

model.add(Conv2D(32, 3, padding='same', activation='relu')) 
model.add(MaxPooling2D(padding='same'))

model.add(Conv2D(64, 3, padding='same', activation='relu')) 
model.add(MaxPooling2D(padding='same')) 
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, activation='relu')) 
model.add(Dense(33 , activation = 'softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds,batch_size=64,epochs=20,validation_data=val_ds)

In [ ]:
model.evaluate(train_ds),model.evaluate(val_ds)